In [19]:
from qdrant_client.http import exceptions, models
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from qdrant_client import QdrantClient
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from operator import itemgetter
from langchain_openai import OpenAIEmbeddings
from typing import Optional
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from qdrant_client.http import models as qdrant_models
from langchain_core.runnables import RunnableBranch

In [20]:
load_dotenv()

True

In [21]:
prompt = """
# 角色
你是一名ESG（環境、社會和治理）專家，可以提供具體公司的ESG行為和成績的相關資訊。

## 技能
### 技能1：提供環境資訊
- 根據使用者查詢的公司，從您的知識庫中搜索並提供該公司的環境績效和政策。

### 技能2：提供社會責任資訊
- 根據使用者查詢的公司，搜索並回答該公司的社會責任項目。

### 技能3：提供治理結構資訊
- 根據使用者查詢的公司，搜索並提供該公司的治理結構和政策。

## 使用提示：
- 必須問關於公司ESG行為的問題。
- 必須使用使用者所用的語言來回答問題。
- 必須使用以下問題格式：
  1. 「請問[公司名稱]在環境方面有哪些主要措施？」
  2. 「[公司名稱]的社會責任項目包括什麼？」
  3. 「關於[公司名稱]的治理結構，您能提供哪些資訊？」
- 好好使用您的知識庫，根據使用者的問題提供信息。
- 目前一次能詢問的公司資料，不能超過時間
- 使用繁體字回覆使用者的問題

## 限制：針對以下內容，無法回覆，但要把不能回覆的原因說明清楚
- 限制一：使用者詢問超過10家公司的ESG資訊
- 限制二：使用者詢問內容不符合上述技能
- 限制三：使用者詢問需要查詢大量的公司資訊，有可能需要查詢超過十家公司資訊

Response the question based on the following reference, if trigger constraint, response the full reason why it is not supported.

{reference}

{query}
"""

In [45]:
def rag_process(
    query: str,
    year: Optional[str] = None,
    co_id: Optional[str] = None,
    show_retrieve_docs = False
):

    
    # prompt: response prompt
    response_prompt = ChatPromptTemplate.from_template(template=prompt)


    ## LLM
    model = ChatOpenAI(model="gpt-3.5-turbo")
    client = QdrantClient(url="34.80.177.36:6333")
    collection_name = "greenhousegas"
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    qdrant = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embeddings,
        content_payload_key="content",
    )
    document_filter = {"year": year, "co_id": co_id}
    document_filter = {k: v for k, v in document_filter.items() if v is not None}
    
    # parser
    str_parser = StrOutputParser()
    # 👎 cannot work
    """
    retriever = qdrant.as_retriever(
        search_kwargs={"filter":{"co_id":"2330"}}
    )
    retriever = qdrant.as_retriever(search_type="similarity_score_threshold", search_kwargs={'filter': document_filter, 'score_threshold': 0.1})
    retriever = qdrant.as_retriever(search_type="mmr", search_kwargs={'filter': document_filter, 'k': 5})
    """

    field_conditions = [
        qdrant_models.FieldCondition(
            key=key,
            match=qdrant_models.MatchValue(value=value)
        )
        for key, value in document_filter.items()
    ]

    retriever = qdrant.as_retriever(
                search_kwargs=dict(
                    filter=qdrant_models.Filter(
                        must=field_conditions
                    )
                )
            )
    if show_retrieve_docs:
        retriever_answer = retriever.get_relevant_documents(query)
        print(retriever_answer, len(retriever_answer))
    chain = {"reference": retriever, "query": RunnablePassthrough()} | response_prompt | model | str_parser
    return chain.invoke(query)

In [ ]:
rag_process(query="中鋼企業(直、間接)溫室氣體排放量直接排放量(category1)多少") #會retrieve不到好的資料

In [ ]:
rag_process(query="台積電溫室氣體排放量直接排放量(category1)多少", year="110", co_id="2330") #不合範疇



In [ ]:
rag_process(query="台積電的環境政策", year="110", co_id="2330") # 可以搜到台積電資料